In [6]:
import snowradar
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal
C = 299792458 #Vacuum speed of light
ER_ICE = 3.15

In [2]:
radar_file = "./data/awi/Data_20170410_01_006.mat"
radar_dat = snowradar.AWI(radar_file, l_case='full')

In [61]:
max_elev = radar_dat.elevation.max()
print(max_elev)

66.6986570750567


In [62]:
time_fast = np.transpose(radar_dat.time_fast)

In [63]:
t = radar_dat.elevation - radar_dat.surface * C / 2

In [64]:
tt = (time_fast[-1]- radar_dat.surface) * C/2/np.sqrt(ER_ICE)

In [69]:
min_elev = (t-tt).min()
print(min_elev)

-26.03225548396751


In [71]:
dt = time_fast[1]-time_fast[0]
dr = dt * C/2 / np.sqrt(ER_ICE)

In [75]:
elev_axis = np.arange(max_elev,min_elev,-dr)

In [6]:
radar_sub = np.transpose(radar_dat.data_radar)[:,0:300]


In [77]:
zero_pad_len = len(elev_axis) - len(time_fast)


In [79]:
radar_data = np.transpose(radar_dat.data_radar)

In [84]:
np.zeros([zero_pad_len,radar_data.shape[1]])

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [82]:
radar_data.shape[1]

5868

In [88]:
radar_data_new = np.concatenate((radar_data,np.zeros([zero_pad_len,radar_data.shape[1]])), axis = 0)

In [89]:
radar_data_new.shape

(17025, 5868)

In [126]:
dRange = max_elev - radar_dat.elevation
dBins = np.round(dRange / (C/2) / dt)
dtime = dRange/(C/2);
dt_air = dr/(C/2)
dt_ice = dr/(C/2/np.sqrt(ER_ICE))

In [103]:
radar_data_new.shape[1]

5868

In [123]:
for rline in np.arange(0, radar_data_new.shape[1]):
    surf_elev = radar_dat.elevation[rline] - radar_dat.surface[rline] * C/2;
    time0 = -(max_elev - radar_dat.elevation[rline])/(C/2);
    last_air_idx = (np.argwhere(elev_axis> surf_elev)).max()
    new_time = time0+dt_air*np.arange(0,last_air_idx-1)

In [129]:
if (last_air_idx < len(elev_axis)):
    first_ice_idx = last_air_idx + 1
    time0 = radar_dat.surface[rline] + (surf_elev - elev_axis[first_ice_idx])/(C/2/np.sqrt(ER_ICE))
    #new_time = cat(1,new_time, (time0 + dt_ice*(0:length(elev_axis)-length(new_time)-1)).')

In [110]:
surf_elev

array([-1.01492678])

In [120]:
np.arange(0,last_air_idx-1)

array([    0,     1,     2, ..., 12427, 12428, 12429], dtype=int64)

In [114]:
elev_axis.shape

(17025,)

In [ ]:
new_time = (time0 + dt_air*(0:last_air_idx-1)).';